In [ ]:
"""
Test with datascience bowl 2018
"""

In [8]:
"""
Image augumentation with imgaug
"""
import os
import numpy as np
from skimage.io import imread, imshow
from skimage.transform import resize

## SETTINGS
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
# Training and testing data path
# TRAIN_PATH = "./data/stage1_train/"
# TEST_PATH = "./data/stage1_test/"
TRAIN_PATH = "../../data-science-bowl-2018/train/"
TEST_PATH = "../../data-science-bowl-2018/test1/"

# Get the ids
train_ids = next(os.walk(TRAIN_PATH))[1]
# Set the dimensions of the of the training data, we have 670 images
# Dimensions: (670, 256, 256, 3) 
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

for n, id_ in enumerate(train_ids):
    img = imread(TRAIN_PATH + id_ + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    # Make sure the image is 256x256, we can change this value above by the constant
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    # Prepare the corresponding maskss
    # shape 256x256x1 (only black and white so one channel)
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    # For each mask, resize the image to 256x256, then add a color channel resulting in a (256,265,1) matrix.
    # Then, add the resized matrix image to the final combined training mask by taking the maximum value of
    # the two matrices or in other words, where the mask , resulting in the expected combined mask.
    for mask_file in next(os.walk(TRAIN_PATH + id_ + '/masks/'))[2]:
        mask_ = imread(TRAIN_PATH + id_ + '/masks/' + mask_file)
        mask_ = resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        mask_ = np.expand_dims(mask_, axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask

<ipython-input-8-a135c07aa60e>:24: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
<ipython-input-8-a135c07aa60e>:33: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)


In [9]:
import imgaug as ia
from imgaug import augmenters as iaa

vert_flipper = iaa.Fliplr(1.0)
horizontal_flipper = iaa.Flipud(1.0)
X_train_aug = np.zeros((len(train_ids) * 4, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train_aug = np.zeros((len(train_ids) * 4, IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

for i in range(0, len(X_train)):
  X_train_aug[i * 4] = X_train[i]
  Y_train_aug[i * 4] = Y_train[i]
  X_train_aug[i * 4 + 1] = vert_flipper.augment_image(X_train[i])
  Y_train_aug[i * 4 + 1] = vert_flipper.augment_image(Y_train[i])
  X_train_aug[i * 4 + 2] = horizontal_flipper.augment_image(X_train[i])
  Y_train_aug[i * 4 + 2] = horizontal_flipper.augment_image(Y_train[i])
  X_train_aug[i * 4 + 3] = vert_flipper.augment_image(horizontal_flipper.augment_image(X_train[i]))
  Y_train_aug[i * 4 + 3] = vert_flipper.augment_image(horizontal_flipper.augment_image(Y_train[i]))

<ipython-input-9-58484ecd23f4>:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y_train_aug = np.zeros((len(train_ids) * 4, IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
